# NBA Hall of Fame Predicor 

In [97]:
import pandas as pd 
import numpy as np

In [98]:
df = pd.read_csv('data/PlayerStatistics.csv')

/var/folders/v5/pt9xf7710kscw08ztc5n1_h40000gn/T/ipykernel_19029/2883235819.py:1: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/PlayerStatistics.csv')


In [99]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1637420 entries, 0 to 1637419
Data columns (total 35 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   firstName                1637420 non-null  object 
 1   lastName                 1637420 non-null  object 
 2   personId                 1637420 non-null  int64  
 3   gameId                   1637420 non-null  int64  
 4   gameDateTimeEst          1637420 non-null  object 
 5   playerteamCity           1637420 non-null  object 
 6   playerteamName           1637420 non-null  object 
 7   opponentteamCity         1637420 non-null  object 
 8   opponentteamName         1637420 non-null  object 
 9   gameType                 1629175 non-null  object 
 10  gameLabel                94707 non-null    object 
 11  gameSubLabel             6166 non-null     object 
 12  seriesGameNumber         135048 non-null   float64
 13  win                      1637420 non-null 

In [100]:
# Data Cleaning 
df = df[['firstName', 'lastName', 'points', 'personId', 'numMinutes', 'assists', 'reboundsTotal', 'turnovers', 'gameId', 'gameType', 'blocks', 'steals', 'gameDateTimeEst', 'fieldGoalsAttempted', 'fieldGoalsMade']]
df = df[df['numMinutes'] > 0.0]
df = df[df['gameType'].isin(['Regular Season', 'in-season'])]

In [101]:
df['gameDateTimeEst'] = pd.to_datetime(df['gameDateTimeEst'], utc=True)
df['gameDateTimeEst'].min(), df['gameDateTimeEst'].max()
df['gameDateTimeEst'].dt.year.unique()

array([2025, 2024, 2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015,
       2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004,
       2003, 2002, 2001, 2000, 1999, 1998, 1997, 1996, 1995, 1994, 1993,
       1992, 1991, 1990, 1989, 1988, 1987, 1986, 1985, 1984, 1983, 1982,
       1981, 1980, 1979, 1978, 1977, 1976, 1975, 1974, 1973, 1972, 1971,
       1970, 1969, 1968, 1967, 1966, 1965, 1964, 1963, 1962, 1961, 1960,
       1959, 1958, 1957, 1956, 1955, 1954, 1953, 1952, 1951])

In [102]:
numGamesPlayedDf = df.groupby('personId')['gameId'].nunique().reset_index()
numGamesPlayedDf = numGamesPlayedDf.rename(columns={'gameId': 'numGamesPlayed'})

lastYearPlayedDf = df.groupby('personId')['gameDateTimeEst'].agg(['max']).reset_index()

In [103]:
aggregatedData = df.groupby(['personId', 'firstName', 'lastName'])[['points', 'reboundsTotal', 'assists','blocks', 'steals', 'fieldGoalsAttempted', 'fieldGoalsMade']].sum().reset_index()
aggregatedData = aggregatedData.merge(numGamesPlayedDf, on='personId', how = 'left')
aggregatedData['fieldGoalPercentage'] = (aggregatedData['fieldGoalsMade'] / aggregatedData['fieldGoalsAttempted'].replace(0, np.nan))
aggregatedData = aggregatedData[aggregatedData['numGamesPlayed'] > 5]

# add averages for PRA as well
aggregatedData['pointsPerGame'] = (aggregatedData['points'] / aggregatedData['numGamesPlayed'])
aggregatedData['assistsPerGame'] = (aggregatedData['assists'] / aggregatedData['numGamesPlayed'])
aggregatedData['reboundsPerGame'] = (aggregatedData['reboundsTotal'] / aggregatedData['numGamesPlayed'])


# need to make a cutoff so only players who finished their careers 10 years ago included in training data (those players are likely to have been considered in multiple ballots)
aggregatedData = aggregatedData.merge(lastYearPlayedDf, on = 'personId', how='left')
aggregatedData = aggregatedData[aggregatedData['max'].dt.year <= 2015]


trainingData = aggregatedData[['firstName', 'lastName', 'points', 'reboundsTotal', 'assists', 'pointsPerGame', 'assistsPerGame', 'reboundsPerGame', 'blocks', 'steals', 'fieldGoalPercentage', 'numGamesPlayed']]
trainingData


,firstName,lastName,points,reboundsTotal,assists,pointsPerGame,assistsPerGame,reboundsPerGame,blocks,steals,fieldGoalPercentage,numGamesPlayed
0,Byron,Scott,15097.0,2987.0,2727.0,14.069897,2.541473,2.783784,276.0,1222.0,0.482393,1073
1,Grant,Long,9518.0,6154.0,1716.0,9.489531,1.710867,6.135593,366.0,1199.0,0.466667,1003
2,Dan,Schayes,8751.0,5651.0,1295.0,7.737401,1.145004,4.996463,826.0,525.0,0.480813,1131
3,Sedale,Threatt,9328.0,1709.0,3613.0,9.808623,3.799159,1.797056,136.0,1139.0,0.484706,951
4,Chris,King,693.0,311.0,116.0,6.728155,1.126214,3.019417,34.0,74.0,0.422777,103
...,...,...,...,...,...,...,...,...,...,...,...,...
3289,George,Mikan,282.0,130.0,31.0,14.842105,1.631579,6.842105,0.0,0.0,0.454128,19
3290,Rick,Barry,18270.0,5135.0,3953.0,23.273885,5.035669,6.541401,227.0,1029.0,0.451328,785
3291,Artis,Gilmore,14947.0,8778.0,1667.0,17.004551,1.896473,9.986348,1553.0,360.0,0.599541,879
3292,Oscar,Robertson,19135.0,5443.0,7178.0,24.532051,9.202564,6.978205,4.0,22.0,0.491813,780


In [104]:
# Add in career accolades 
awardsDf = pd.read_csv("data/NBA_players_accolades.csv")
awardsDf = awardsDf[['Player', 'All Star', 'All NBA', 'All Defensive', 'DPOY', 'Finals MVP', 'MVP', 'Championships']]
awardsDf['Player'] = awardsDf['Player'].str.replace('*', '', regex=False)

awardsDf['firstName'] = awardsDf['Player'].apply(lambda s: s.split(' ', 1)[0])
awardsDf['lastName']  = awardsDf['Player'].apply(lambda s: s.split(' ', 1)[1] if ' ' in s else '')


awardsDf = awardsDf[['firstName', 'lastName', 'All Star', 'All NBA', 'All Defensive', 'DPOY', 'Finals MVP', 'MVP', 'Championships']]

# merge this with the training set

trainingData = trainingData.merge(awardsDf, on=['firstName', 'lastName'], how='left')
trainingData[['All Star', 'All NBA', 'All Defensive', 'DPOY', 'Finals MVP', 'MVP', 'Championships']] = trainingData[['All Star', 'All NBA', 'All Defensive', 'DPOY', 'Finals MVP', 'MVP', 'Championships']].fillna(0)
trainingData

,firstName,lastName,points,reboundsTotal,assists,pointsPerGame,assistsPerGame,reboundsPerGame,blocks,steals,fieldGoalPercentage,numGamesPlayed,All Star,All NBA,All Defensive,DPOY,Finals MVP,MVP,Championships
0,Byron,Scott,15097.0,2987.0,2727.0,14.069897,2.541473,2.783784,276.0,1222.0,0.482393,1073,0.0,0.0,0.0,0.0,0.0,0.0,3.0
1,Grant,Long,9518.0,6154.0,1716.0,9.489531,1.710867,6.135593,366.0,1199.0,0.466667,1003,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Dan,Schayes,8751.0,5651.0,1295.0,7.737401,1.145004,4.996463,826.0,525.0,0.480813,1131,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Sedale,Threatt,9328.0,1709.0,3613.0,9.808623,3.799159,1.797056,136.0,1139.0,0.484706,951,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Chris,King,693.0,311.0,116.0,6.728155,1.126214,3.019417,34.0,74.0,0.422777,103,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2889,George,Mikan,282.0,130.0,31.0,14.842105,1.631579,6.842105,0.0,0.0,0.454128,19,4.0,6.0,0.0,0.0,0.0,0.0,5.0
2890,Rick,Barry,18270.0,5135.0,3953.0,23.273885,5.035669,6.541401,227.0,1029.0,0.451328,785,12.0,6.0,0.0,0.0,1.0,0.0,1.0
2891,Artis,Gilmore,14947.0,8778.0,1667.0,17.004551,1.896473,9.986348,1553.0,360.0,0.599541,879,11.0,0.0,5.0,0.0,0.0,1.0,1.0
2892,Oscar,Robertson,19135.0,5443.0,7178.0,24.532051,9.202564,6.978205,4.0,22.0,0.491813,780,12.0,11.0,0.0,0.0,0.0,1.0,1.0


In [105]:
# add in HOF status now:
hofDf = pd.read_csv("data/NBA_PLAYERS_hof.csv")
hofDf = hofDf[['Name', 'HOF']]

hofDf['firstName'] = hofDf['Name'].apply(lambda s: s.split(' ', 1)[0])
hofDf['lastName']  = hofDf['Name'].apply(lambda s: s.split(' ', 1)[1] if ' ' in s else '')

hofDf = hofDf[['firstName', 'lastName', 'HOF']]

trainingData = trainingData.merge(hofDf, on=['firstName', 'lastName'], how='left')

trainingData

,firstName,lastName,points,reboundsTotal,assists,pointsPerGame,assistsPerGame,reboundsPerGame,blocks,steals,fieldGoalPercentage,numGamesPlayed,All Star,All NBA,All Defensive,DPOY,Finals MVP,MVP,Championships,HOF
0,Byron,Scott,15097.0,2987.0,2727.0,14.069897,2.541473,2.783784,276.0,1222.0,0.482393,1073,0.0,0.0,0.0,0.0,0.0,0.0,3.0,False
1,Grant,Long,9518.0,6154.0,1716.0,9.489531,1.710867,6.135593,366.0,1199.0,0.466667,1003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
2,Dan,Schayes,8751.0,5651.0,1295.0,7.737401,1.145004,4.996463,826.0,525.0,0.480813,1131,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,Sedale,Threatt,9328.0,1709.0,3613.0,9.808623,3.799159,1.797056,136.0,1139.0,0.484706,951,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
4,Chris,King,693.0,311.0,116.0,6.728155,1.126214,3.019417,34.0,74.0,0.422777,103,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3036,George,Mikan,282.0,130.0,31.0,14.842105,1.631579,6.842105,0.0,0.0,0.454128,19,4.0,6.0,0.0,0.0,0.0,0.0,5.0,True
3037,Rick,Barry,18270.0,5135.0,3953.0,23.273885,5.035669,6.541401,227.0,1029.0,0.451328,785,12.0,6.0,0.0,0.0,1.0,0.0,1.0,True
3038,Artis,Gilmore,14947.0,8778.0,1667.0,17.004551,1.896473,9.986348,1553.0,360.0,0.599541,879,11.0,0.0,5.0,0.0,0.0,1.0,1.0,True
3039,Oscar,Robertson,19135.0,5443.0,7178.0,24.532051,9.202564,6.978205,4.0,22.0,0.491813,780,12.0,11.0,0.0,0.0,0.0,1.0,1.0,True


In [106]:
# some EDA
numHofPlayers = trainingData[trainingData['HOF'] == True].shape[0]
numNotHofPlayers = trainingData[trainingData['HOF'] == False].shape[0]
print(f"number of HOF players: {numHofPlayers}")
print(f"number of non-HOF players: {numNotHofPlayers}")
print(f"HOF players make up {((numHofPlayers / (numNotHofPlayers + numHofPlayers))*100):.2f}% of players")

number of HOF players: 160
number of non-HOF players: 2670
HOF players make up 5.65% of players


# Machine Learning Models

In [107]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report,
    roc_auc_score, 
    average_precision_score
)
from sklearn.model_selection import StratifiedKFold

In [108]:
trainingData.isna().sum()
labeled = trainingData[trainingData['HOF'].notna()].copy()
labeled['HOF'] = labeled['HOF'].astype(int)  # False -> 0, True -> 1
labeled = labeled.dropna(subset=['HOF'])



In [109]:
# Model #1: Logistic Regression 

X = labeled[['points', 'reboundsTotal', 'assists', 'pointsPerGame', 'assistsPerGame', 'reboundsPerGame', 'blocks', 'steals', 'fieldGoalPercentage', 'numGamesPlayed', 'All Star', 'All NBA', 'All Defensive', 'DPOY', 'Finals MVP', 'MVP', 'Championships']]
y = labeled['HOF']


skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=23)



precisionResults = []
recallResults = []
f1results = []
roc_aucs = []
pr_aucs = []

for trainingIndex, testIndex in skf.split(X, y):
    X_train, X_test = X.iloc[trainingIndex], X.iloc[testIndex]
    y_train, y_test = y.iloc[trainingIndex], y.iloc[testIndex]

    model = LogisticRegression(max_iter=5000, class_weight='balanced')
    model.fit(X_train, y_train)

    predictions = model.predict(X_test)
    yProb = model.predict_proba(X_test)[:, 1]
    precisionResults.append(precision_score(y_test, predictions))
    recallResults.append(recall_score(y_test, predictions))
    f1results.append(f1_score(y_test, predictions))
    roc_aucs.append(roc_auc_score(y_test, predictions))
    pr_aucs.append(average_precision_score(y_test, predictions))




print("-------LOGISTIC REGRESSION RESULTS-------")
print("Precision:", np.mean(precisionResults), "±", np.std(precisionResults))
print("Recall:", np.mean(recallResults), "±", np.std(recallResults))
print("F1:", np.mean(f1results), "±", np.std(f1results))
print("ROC-AUC:", np.mean(roc_aucs), "±", np.std(roc_aucs))
print("PR-AUC:", np.mean(pr_aucs), "±", np.std(pr_aucs))


-------LOGISTIC REGRESSION RESULTS-------
Precision: 0.5336134601832276 ± 0.03565994550844816
Recall: 0.84375 ± 0.08149003006503311
F1: 0.6517536231884058 ± 0.03944948833387657
ROC-AUC: 0.8995903558052435 ± 0.038802915503129566
PR-AUC: 0.4593603162889683 ± 0.05073817307294949


In [110]:
from sklearn.ensemble import RandomForestClassifier

In [111]:
# random forest model

precisionResults = []
recallResults = []
f1results = []
roc_aucs = []
pr_aucs = []



for trainingIndex, testIndex in skf.split(X, y):
    X_train, X_test = X.iloc[trainingIndex], X.iloc[testIndex]
    y_train, y_test = y.iloc[trainingIndex], y.iloc[testIndex]

    newModel = RandomForestClassifier(n_estimators=300, max_depth=None, min_samples_leaf=5, class_weight='balanced', random_state=23, n_jobs=1)

    newModel.fit(X_train, y_train)

    prediction = newModel.predict(X_test)
    probability = newModel.predict_proba(X_test)[:, 1]

    precisionResults.append(precision_score(y_test, prediction))
    recallResults.append(recall_score(y_test, prediction))
    f1results.append(f1_score(y_test, prediction))
    roc_aucs.append(roc_auc_score(y_test, probability))
    pr_aucs.append(average_precision_score(y_test, probability))


print("-------Random Forest Results-------")
print("Precision:", np.mean(precisionResults), "±", np.std(precisionResults))
print("Recall:", np.mean(recallResults), "±", np.std(recallResults))
print("F1:", np.mean(f1results), "±", np.std(f1results))
print("ROC-AUC:", np.mean(roc_aucs), "±", np.std(roc_aucs))
print("PR-AUC:", np.mean(pr_aucs), "±", np.std(pr_aucs))


-------Random Forest Results-------
Precision: 0.745823396926338 ± 0.07123200843660567
Recall: 0.78125 ± 0.06555055301063448
F1: 0.7625502939089897 ± 0.06499775361041686
ROC-AUC: 0.9662219101123597 ± 0.020084578291915577
PR-AUC: 0.8328860548879536 ± 0.06899864717956557


In [112]:
from xgboost import XGBClassifier 

In [113]:
mask = X.notna().all(axis=1) & y.notna()
X = X[mask]
y = y[mask]
X = X.reset_index(drop=True)
y = y.reset_index(drop=True)

In [114]:
precisionResults = []
recallResults = []
f1results = []
roc_aucs = []
pr_aucs = []

numPos = np.sum(y == 1)
numNeg = np.sum(y == 0)
weight = numNeg / numPos

featureImportance = []

for trainingIndex, testIndex in skf.split(X, y):
    X_train = X.iloc[trainingIndex].values
    X_test  = X.iloc[testIndex].values
    y_train = y.iloc[trainingIndex].values
    y_test  = y.iloc[testIndex].values
    model = XGBClassifier( n_estimators=300,max_depth=4, learning_rate=0.05,subsample=0.8,colsample_bytree=0.8,scale_pos_weight=weight,eval_metric="logloss",random_state=23,n_jobs=-1)

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]

    precisionResults.append(precision_score(y_test, y_pred))
    recallResults.append(recall_score(y_test, y_pred))
    f1results.append(f1_score(y_test, y_pred))
    roc_aucs.append(roc_auc_score(y_test, y_prob))
    pr_aucs.append(average_precision_score(y_test, y_prob))

    featureImportance.append(pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False))

feature = pd.concat(featureImportance, axis=1)
mean = feature.mean(axis=1)
std  = feature.std(axis=1)

featureSummary = (
    pd.DataFrame({
        "Mean Importance": mean
    })
    .sort_values("Mean Importance", ascending=False)
)

print("-------XGBoost Results-------")
print("Precision:", np.mean(precisionResults), "±", np.std(precisionResults))
print("Recall:", np.mean(recallResults), "±", np.std(recallResults))
print("F1:", np.mean(f1results), "±", np.std(f1results))
print("ROC-AUC:", np.mean(roc_aucs), "±", np.std(roc_aucs))
print("PR-AUC:", np.mean(pr_aucs), "±", np.std(pr_aucs))
print(featureSummary.head(10))



-------XGBoost Results-------
Precision: 0.7451177633240951 ± 0.04847964925423347
Recall: 0.775 ± 0.08244316223920575
F1: 0.7569345842890426 ± 0.05034314475648657
ROC-AUC: 0.9612827715355804 ± 0.02558605594592397
PR-AUC: 0.8170360858056709 ± 0.0799091611179787
                 Mean Importance
All Star                0.365340
All NBA                 0.123640
Championships           0.058665
DPOY                    0.054132
blocks                  0.049126
assistsPerGame          0.047343
steals                  0.045213
pointsPerGame           0.041568
reboundsPerGame         0.032521
points                  0.032091


In [115]:
X_train = X
y_train = y

numPos = np.sum(y_train == 1)
numNeg = np.sum(y_train == 0)
weight = numNeg / numPos

final_model = XGBClassifier(
    n_estimators=300,
    max_depth=4,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=weight,
    eval_metric="logloss",
    random_state=23,
    n_jobs=-1
)

final_model.fit(X_train.values, y_train.values)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.05, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=4,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=300,
              n_jobs=-1, num_parallel_tree=None, random_state=23, ...)

In [137]:
# Latest season year in your dataset (e.g., 2025)
latest_year = int(aggregatedAll['max'].dt.year.max())
eligibility_year = latest_year - 3  # eligible if retired <= this year

# Attach retirement year to hofDf using names (since hofDf has no personId)
hof_with_retirement = hofDf.merge(
    aggregatedAll[['firstName', 'lastName', 'max']],
    on=['firstName', 'lastName'],
    how='left'
)

# Define "already inducted / should be excluded from future list" as:
# - labeled HOF True
# - and retired at least 3 years ago (eligible by now)
exclude_names = set(
    hof_with_retirement[
        (hof_with_retirement['HOF'] == True) &
        (hof_with_retirement['max'].dt.year <= eligibility_year)
    ][['firstName', 'lastName']]
    .apply(tuple, axis=1)
)

# Future candidates: active or recent players
future_candidates = aggregatedAll[
    aggregatedAll['max'].dt.year > 2015
].copy()

# Remove the already-inducted/eligible HOF names
future_candidates = future_candidates[
    ~future_candidates[['firstName', 'lastName']]
    .apply(tuple, axis=1)
    .isin(exclude_names)
].copy()

# Predict Hall of Fame probabilities
future_candidates[X.columns] = future_candidates[X.columns].fillna(0)
X_future = future_candidates[X.columns].values
future_candidates['HOF_Probability'] = final_model.predict_proba(X_future)[:, 1]

# Top 5 projected future Hall of Famers
top5_future = (
    future_candidates
    .sort_values('HOF_Probability', ascending=False)
    .head(5)
)

top5_future[['firstName', 'lastName', 'HOF_Probability']]


,firstName,lastName,HOF_Probability
958,LeBron,James,0.999723
2801,Kevin,Durant,0.999573
2929,Stephen,Curry,0.999340
2641,Chris,Paul,0.998525
3054,Kyrie,Irving,0.998031
